# Consumer Opt. with PV

In [89]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating project at `C:\Users\ESchr\workspace\uni\ATIS3\Single_Project\ATIES3_single_project_ES`


In [90]:
using JuMP, LinearAlgebra, StatsPlots, Statistics, CSV, DataFrames, XLSX, CPLEX, AxisArrays

## Input handling

In [101]:
input_path = "input\\"
xl = XLSX.readxlsx(input_path * "input.xlsx") 
data = Dict()

for s in XLSX.sheetnames(xl)
    data[s] = DataFrame(XLSX.gettable(xl[s]; infer_eltypes=true)...)
end

#Rename column from gams export

## Create Axis-Arrays

In [102]:
W_PP = data["pool_price_scenarios"][:, :Scenario]
W_NAM = data["Non_anticipativity_matrix"][:, :Scenario]
B = data["bilateral_contract_data"][:, :Contract]
D = data["electricity_demand"][:, :Demand]
T = ["t1", "t2", "t3", "t4"]
println("Sets created")

Sets created


In [103]:
Dpool_price_scenarios = Dict(c => AxisArray(data["pool_price_scenarios"][:, c], w_pp=W_PP) for c in names(data["pool_price_scenarios"]));
Dpool_price_scenarios["t1"]["w1"]

Dbilateral_contract_data = Dict(c => AxisArray(data["bilateral_contract_data"][:, c], b=B) for c in names(data["bilateral_contract_data"]));
Dbilateral_contract_data["price"]["b1"]

Delectricity_demand = Dict(c => AxisArray(data["electricity_demand"][:, c], d=D) for c in names(data["electricity_demand"]));
Delectricity_demand["t1"]["d1"]

Dnon_anticipativity_matrix = Dict(c => AxisArray(data["Non_anticipativity_matrix"][:, c], w_nam=W_NAM) for c in names(data["Non_anticipativity_matrix"]));
Dnon_anticipativity_matrix["k1"]["w1"]
println("Axis Arrays created")

Axis Arrays created


In [106]:
#Sets
B = []#Bilateral contracts
K = []#Stages

W = []#Scenarios
T = []#Periods
N = [1,2,3,4]#Blocks in the piecewise linear production cost

#Bt[B,T]#Mapping of bilateral contracts active in period T
#Kb[B,K] Mapping of decisions on bilateral contracts made in stage K;

# Risk
β = 0.0
α = 0.95

0.95

In [107]:
co_pv = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [108]:
@variables(co_pv, begin
        z #Objective function value
        PB[B,W] #Power purchased from bilateral contracts
        s[B,W], Bin #Selection of bilateral contracts
        EG[N,T,W] >= 0 #Energy produced by the self-production unit
        EP[T,W] #Energy traded in the pool
        
        η[W] >= 0 #Auxiliary variable used to calculate the CVaR
        ζ #Auxiliary variable used to calculate the CVaR
    end)

(z, 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{VariableRef}, 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{VariableRef}, 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
    Dimension 2, Any[]
    Dimension 3, Any[]
And data, a 4×0×0 Array{VariableRef, 3}, 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{VariableRef}, 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, Any[]
And data, a 0-element Vector{VariableRef}, ζ)

In [109]:
@constraints(co_pv, begin
        LimitContractMin[b in B, w in W], PB[b,w] >= Dbilateral_contract_data["min_power"][b]*s[b,w]
        LimitContractMax[b in B, w in W], PB[b,w] <= Dbilateral_contract_data["max_power"][b]*s[B,W]
        

        EnergyBalance[t in T, w in W], sum(EG[t,w] for n in N) + EP[t,w] + sum(PB[b,w]*d[t] for b in B)#=$Bt(B,T)=# == Delectricity_demand["t_1"]["d1"]#-EPC[t]
        #Arbitrage[t in T, w in W], sum(EG[n,t,w] for n in N) >= -EP[t,w]
        #Non_Anticipativity $(KB(B,K) and (A(W,K)=1) and (ord(W) lt card(W))
        #Non_Anticipativity[b in B, w in W, k in K], PB[b,w] == PB[b, w+1]
        #CVaRconstr[w in W], sum(sum( λ_B[b,t,w]*PB[b,w]*d[t] for b in B #=B$Bt(B,T)=#) + λ_P[w,t]*EP[t,w] + sum(SG[n]*EG[n,t,w] for n in N) for t in T)- ζ - η[w] == 0        
        
        
        #attetion hard coded
        #SelfProduction1[nl in 1:1, t in T, w in W], EG[nl,t,w] <= EGT[nl]
        #SelfProduction[nl in 2:NL, t in T, w in W], EG[nl,t,w] <= EGT[nl] - EGT[nl-1]
    end)

(2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{Any}, 2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{Any}, 2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any[]
    Dimension 2, Any[]
And data, a 0×0 Matrix{Any})

In [110]:
co_pv

A JuMP Model
Feasibility problem with:
Variables: 2
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: EG, EP, EnergyBalance, LimitContractMax, LimitContractMin, PB, s, z, ζ, η